In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sunpy
from sunpy.map import Map
from sunpy.instr.aia import aiaprep

# Load original image
origim = Map("aia171_2014-01-11T00_00.fits")
print origim.scale, origim.shape
lowx, highx = (origim.xrange[0] / origim.scale['x'],
               origim.xrange[1] / origim.scale['x'])
lowy, highy = (origim.yrange[0] / origim.scale['y'],
               origim.yrange[1] / origim.scale['y'])
x_grid, y_grid = np.mgrid[lowx:highx-1, lowy:highy-1]
r_grid = np.sqrt((x_grid ** 2.0) + (y_grid ** 2.0))
outer_rad = (origim.rsun_arcseconds * 1.4) / origim.scale['x']
origim.data[r_grid > outer_rad] = 1

# Process original image with SunPy
pypreppedim = aiaprep(origim)
print pypreppedim.scale, pypreppedim.shape
lowx, highx = (pypreppedim.xrange[0] / pypreppedim.scale['x'],
               pypreppedim.xrange[1] / pypreppedim.scale['x'])
lowy, highy = (pypreppedim.yrange[0] / pypreppedim.scale['y'],
               pypreppedim.yrange[1] / pypreppedim.scale['y'])
x_grid, y_grid = np.mgrid[lowx:highx-1, lowy:highy-1]
r_grid = np.sqrt((x_grid ** 2.0) + (y_grid ** 2.0))
outer_rad = (pypreppedim.rsun_arcseconds * 1.4) / pypreppedim.scale['x']
pypreppedim.data[r_grid > outer_rad] = 1

# Load image processed with IDL
idlpreppedim = Map("aia171_2014-01-11T00_00_idlprepped.fits")
print idlpreppedim.scale, idlpreppedim.shape
lowx, highx = (idlpreppedim.xrange[0] / idlpreppedim.scale['x'],
               idlpreppedim.xrange[1] / idlpreppedim.scale['x'])
lowy, highy = (idlpreppedim.yrange[0] / idlpreppedim.scale['y'],
               idlpreppedim.yrange[1] / idlpreppedim.scale['y'])
x_grid, y_grid = np.mgrid[lowx:highx-1, lowy:highy-1]
r_grid = np.sqrt((x_grid ** 2.0) + (y_grid ** 2.0))
outer_rad = (idlpreppedim.rsun_arcseconds * 1.4) / idlpreppedim.scale['x']
idlpreppedim.data[r_grid > outer_rad] = 1

In [ ]:
"""Visually compare original image to image processed using SunPy"""

fig = plt.figure(figsize=(15, 8))

fig.add_subplot(1, 2, 1)
origim.plot()
plt.title('Max: {:.2f}, Mean: {:.2f}, Min: {:.2f}'.format(origim.max(), origim.mean(), origim.min()))
plt.colorbar(orientation='horizontal')

fig.add_subplot(1, 2, 2)
pypreppedim.plot()
plt.title('Max: {:.2f}, Mean: {:.2f}, Min: {:.2f}'.format(pypreppedim.max(), pypreppedim.mean(), pypreppedim.min()))
plt.colorbar(orientation='horizontal')
plt.show()

In [ ]:
"""Assess difference of SunPy image from original."""

diffim1 = Map(pypreppedim.data.copy(), pypreppedim.meta.copy())
diffim1.data = (origim.data - pypreppedim.data) / origim.data
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 8))
plt.sca(ax[0])
meandiff = diffim1.data[np.isfinite(diffim1.data)].mean()
stddiff = diffim1.data[np.isfinite(diffim1.data)].std()
maxdiff = diffim1.data[np.isfinite(diffim1.data)].max()
mindiff = diffim1.data[np.isfinite(diffim1.data)].min()
#diffim1.plot(cmap='coolwarm', vmin=meandiff-(2*stddiff), vmax=meandiff+(2*stddiff))
diffim1.plot(cmap='coolwarm', vmin=-2, vmax=2)
plt.title('Min: {:.2f}, Max{:.2f}\nMean: {:.2f}, Std Dev: {:.2f}'.format(maxdiff, mindiff, meandiff, stddiff))
plt.colorbar(orientation='horizontal')
plt.sca(ax[1])
#plt.hist(diffim1.data.flatten(), range=(meandiff-(10*stddiff), meandiff+(10*stddiff)), bins=50)
plt.hist(diffim1.data.flatten(), range=(-5, 5), bins=50, cumulative=True)
plt.axhline(diffim1.size, linestyle='--', label='Total no. of pixels in image')
plt.axvline(-1, linestyle='--', color='black')
plt.axvline(1, linestyle='--', color='black')
#plt.xlim(-5, 5)
plt.xlabel('(Original pixel value - processed pixel value) / Original pixel value')
plt.ylabel('Pixel count')
#plt.legend()
plt.show()

In [ ]:
print origim.processing_level, pypreppedim.processing_level
print pypreppedim.scale, pypreppedim.shape
print np.nanmin(pypreppedim.data), np.nanmean(pypreppedim.data), np.nanmax(pypreppedim.data)
print pypreppedim.min(), pypreppedim.mean(), pypreppedim.max()
print origim.center, origim.reference_coordinate, origim.reference_pixel
print pypreppedim.center, pypreppedim.reference_coordinate, pypreppedim.reference_pixel

In [ ]:
"""Visually compare original image to image processed using IDL"""

fig = plt.figure(figsize=(15, 8))

print origim.scale, origim.shape
fig.add_subplot(1, 2, 1)
origim.plot()
plt.title('Max: {:.2f}, Mean: {:.2f}, Min: {:.2f}'.format(origim.max(), origim.mean(), origim.min()))
plt.colorbar(orientation='horizontal')

fig.add_subplot(1, 2, 2)
idlpreppedim.plot()
plt.title('Max: {:.2f}, Mean: {:.2f}, Min: {:.2f}'.format(idlpreppedim.max(), idlpreppedim.mean(), idlpreppedim.min()))
plt.colorbar(orientation='horizontal')
plt.show()

In [ ]:
"""Assess difference of IDL image from original."""

diffim2 = Map(idlpreppedim.data.copy(), idlpreppedim.meta.copy())
diffim2.data = (origim.data - idlpreppedim.data) / origim.data
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 8))
plt.sca(ax[0])
meandiff = diffim2.data[np.isfinite(diffim2.data)].mean()
stddiff = diffim2.data[np.isfinite(diffim2.data)].std()
maxdiff = diffim2.data[np.isfinite(diffim2.data)].max()
mindiff = diffim2.data[np.isfinite(diffim2.data)].min()
#diffim2.plot(cmap='coolwarm', vmin=meandiff-(2*stddiff), vmax=meandiff+(2*stddiff))
diffim2.plot(cmap='coolwarm', vmin=-2, vmax=2)
plt.title('Min: {:.2f}, Max{:.2f}\nMean: {:.2f}, Std Dev: {:.2f}'.format(maxdiff, mindiff, meandiff, stddiff))
plt.colorbar(orientation='horizontal')
plt.sca(ax[1])
#plt.hist(diffim2.data.flatten(), range=(meandiff-(10*stddiff), meandiff+(10*stddiff)), bins=50)
plt.hist(diffim2.data.flatten(), range=(-5, 5), bins=50, cumulative=True)
plt.axhline(diffim2.size, linestyle='--', label='Total no. of pixels in image')
plt.axvline(-1, linestyle='--', color='black')
plt.axvline(1, linestyle='--', color='black')
#plt.xlim(-5, 5)
plt.xlabel('(Original pixel value - processed pixel value) / Original pixel value')
plt.ylabel('Pixel count')
#plt.legend()
plt.show()

In [ ]:
print origim.processing_level, idlpreppedim.processing_level
print idlpreppedim.scale, idlpreppedim.shape
print np.nanmin(idlpreppedim.data), np.nanmean(idlpreppedim.data), np.nanmax(idlpreppedim.data)
print idlpreppedim.min(), idlpreppedim.mean(), idlpreppedim.max()
print origim.center, origim.reference_coordinate, origim.reference_pixel
print idlpreppedim.center, idlpreppedim.reference_coordinate, idlpreppedim.reference_pixel

In [ ]:
"""Assess difference of SunPy image from IDL."""

diffim3 = Map(idlpreppedim.data.copy(), idlpreppedim.meta.copy())
diffim3.data = (idlpreppedim.data - pypreppedim.data) / idlpreppedim.data
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 8))
plt.sca(ax[0])
meandiff = diffim3.data[np.isfinite(diffim3.data)].mean()
stddiff = diffim3.data[np.isfinite(diffim3.data)].std()
maxdiff = diffim3.data[np.isfinite(diffim3.data)].max()
mindiff = diffim3.data[np.isfinite(diffim3.data)].min()
#diffim2.plot(cmap='coolwarm', vmin=meandiff-(2*stddiff), vmax=meandiff+(2*stddiff))
diffim3.plot(cmap='coolwarm', vmin=-2, vmax=2)
plt.title('Min: {:.2f}, Max{:.2f}\nMean: {:.2f}, Std Dev: {:.2f}'.format(maxdiff, mindiff, meandiff, stddiff))
plt.colorbar(orientation='horizontal')
plt.sca(ax[1])
#plt.hist(diffim2.data.flatten(), range=(meandiff-(10*stddiff), meandiff+(10*stddiff)), bins=50)
plt.hist(diffim3.data.flatten(), range=(-5, 5), bins=50, cumulative=True)
plt.axhline(diffim3.size, linestyle='--', label='Total no. of pixels in image')
plt.axvline(-1, linestyle='--', color='black')
plt.axvline(1, linestyle='--', color='black')
#plt.xlim(-5, 5)
plt.xlabel('(Original pixel value - processed pixel value) / Original pixel value')
plt.ylabel('Pixel count')
#plt.legend()
plt.show()